In [178]:
import pyopenvdb as vdb
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from IPython.display import clear_output

In [179]:
points = np.load('../data/points.npy')
edges = np.load('../data/edge_array.npy')
edge_radius = np.load('../data/edge_radius.npy')/10
print(np.max(points, axis=0))
std = 5*0.6931

[635.43878174 507.36654663 964.96972656]


In [180]:
def gauss(x):
    return np.exp(-x*x*std)


# make a gaussian integral table
def make_gaussian_integral_table(integral_table_size, x_range):
    table = np.zeros(integral_table_size)
    integral_table_Xrange = x_range
    integral_table_scale = integral_table_size/integral_table_Xrange
    scale = 0.5/(np.sqrt(np.pi/std)*integral_table_scale)
    value1 = gauss(0)
    table[0] = 0.5
    for i in range(1, integral_table_size):
        value2 = gauss(i/integral_table_scale)
        table[i] = table[i-1] + (value1 + value2)*scale
        value1 = value2
    return table

In [181]:
# calculate the distance of a point to a line segment
def distance_to_segment(p, p1, p2):
    v = p2 - p1
    w = p - p1
    c1 = np.dot(w, v)
    if c1 <= 0:
        return np.linalg.norm(p - p1)
    c2 = np.dot(v, v)
    if c2 <= c1:
        return np.linalg.norm(p - p2)
    b = c1 / c2
    pb = p1 + b * v
    return np.linalg.norm(p - pb)

# calculate the projection of a point to a line containing the edge
def project_point_with_parameterization(point, e1, e2):
    E = e2 - e1
    P = point - e1
    alpha = np.dot(P, E) / np.dot(E, E)
    Projection_Point = e1 + alpha * E
    return Projection_Point, alpha






# project the point to the edge, get the new point on the line.
# the edge is defined by two points p1 and p2
# the point is out of the edge
# then rotate the line segment together with the new point to x-axis
# return the rotated p1, p2 and point


def Subtend(p1, p2, point, integral_table, integral_table_size, radius):
    projection, alpha = project_point_with_parameterization(point, p1, p2)
    e1distance = -np.linalg.norm(projection - p1)
    e2distance = np.linalg.norm(projection - p2)
    if alpha < 0:
        e1distance = -e1distance
    elif alpha > 1:
        e2distance = -e2distance
    integral_filter = gauss_integral(e1distance/radius, e2distance/radius, integral_table, integral_table_size)
    return integral_filter

# calculate gaussian convolution given the point and the edge(p1, p2)

def gaussian_convolution(p1, p2, point, integral_table, integral_table_size,radius):
    # distance between the point and the line segment
    proj_point, alpha = project_point_with_parameterization(point,p1,p2)
    proj_p = point - proj_point
    dist = np.linalg.norm(proj_p)
    # dist = distance_to_segment(p1, p2, point)
    dist_filter = gauss(dist/radius)
    integral_filter = Subtend(p1, p2, point, integral_table, integral_table_size, radius)
    if integral_filter > 1.1:
        print(integral_filter)
    # result = dist_filter*integral_filter
    return dist_filter*integral_filter
    

def area(x, integral_table_size, integral_table):
    if x >0:
        area = area_postive_x(x, integral_table_size, integral_table)
        return area
    elif x < 0:
        return 1- area_postive_x(-x, integral_table_size, integral_table)
    else:
        return area_postive_x(x, integral_table_size, integral_table)

        

def area_postive_x(x, integral_table_size, integral_table):
    integral_table_Xrange = 10
    
    integral_table_scale = integral_table_size/integral_table_Xrange
    scaledX = x*integral_table_scale
    # if np.isnan(scaledX):
    #     print(scaledX)
    #     print(x)
        # print(integral_table)
    table_index = min(integral_table_size-1, int(scaledX))
    area = integral_table[table_index]
    if table_index < integral_table_size-1:
        area += (scaledX - table_index)*(integral_table[table_index+1]-area)
    return area    
    
def gauss_integral(x1, x2, integral_table, integral_table_size):
    # integral_table_size = 1000
    # integral_table = make_gaussian_integral_table(integral_table_size)
    return area(max(x2,x1), integral_table_size, integral_table) - area(min(x1,x2), integral_table_size, integral_table)

In [182]:
# define the function to generate the edge bounding box
def generate_edge_bounding_box(points, edges, edge_radius):
    edge_bounding_boxes = []
    for i in range(len(edges)):
        edge = edges[i]
        p0 = points[edge[0]]
        p1 = points[edge[1]]
        radius = edge_radius[i]
        q_0x = int(np.floor(min(p0[0], p1[0]) - radius))
        q_0y = int(np.floor(min(p0[1], p1[1]) - radius))
        q_0z = int(np.floor(min(p0[2], p1[2]) - radius))
        q_1x = int(np.ceil(max(p0[0], p1[0]) + radius))
        q_1y = int(np.ceil(max(p0[1], p1[1]) + radius))
        q_1z = int(np.ceil(max(p0[2], p1[2]) + radius))
        q_0 = np.array([q_0x, q_0y, q_0z])
        q_1 = np.array([q_1x, q_1y, q_1z])
        edge_bounding_boxes.append([q_0, q_1])
    return np.array(edge_bounding_boxes)

# scaling the data to enlarge the resolution
# the resolution of the grid is 2048x2048x2048
def scale_data(points, scale):
    points = points * scale
    return points



In [183]:
vdb_grid = vdb.FloatGrid(0)
grid_accessor = vdb_grid.getAccessor()

In [184]:
def load_edge_bounding_boxes_to_grid(edge_bounding_boxes, grid):
    grid_accessor = grid.getAccessor()
    for edge_bounding_box in edge_bounding_boxes:
        q_0 = edge_bounding_box[0]
        q_1 = edge_bounding_box[1]
        grid.fill(q_0, q_1, float(0))

edge_bounding_boxes = generate_edge_bounding_box(points, edges, edge_radius)
load_edge_bounding_boxes_to_grid(edge_bounding_boxes, vdb_grid)


In [185]:
print(edge_bounding_boxes.shape)
print(vdb_grid.activeVoxelCount())
(q0,q1) = vdb_grid.evalActiveVoxelBoundingBox()
print(q0,q1)

print(np.max(points, axis=0))

(50176, 2, 3)
55094723
(8, 10, 3) (637, 509, 966)
[635.43878174 507.36654663 964.96972656]


In [186]:
def precompute_distances(points, edges, edge_radius, bounding_boxes, grid):
    tem_distance_grid = grid.deepCopy()
    print(tem_distance_grid.evalActiveVoxelBoundingBox())
    grid_accessor = tem_distance_grid.getAccessor()
    bounding_box_grid = grid.deepCopy()
    bounding_box_grid_accessor = bounding_box_grid.getAccessor()
    integral_table_size = 10000
    x_range = 10
    integral_table = make_gaussian_integral_table(integral_table_size,x_range)
    print("integral table made")
    for i in range(len(edges)):
        edge = edges[i]
        p1 = points[edge[0]]
        p2 = points[edge[1]]
        cur_edge_radius = edge_radius[i]
        bounding_box = bounding_boxes[i]

        # np.save('integral_table.txt', integral_table)
        clear_output(wait=True)
        for x in range(bounding_box[0][0], bounding_box[1][0]+1):
            for y in range(bounding_box[0][1], bounding_box[1][1]+1):
                for z in range(bounding_box[0][2], bounding_box[1][2]+1):
                    p = np.array([x, y, z])
                    if grid_accessor.isValueOn(p):
                        value = grid_accessor.getValue(p)
                        new_value = gaussian_convolution(p1,p2,p, integral_table, integral_table_size,cur_edge_radius)
                        value = value + new_value
                        grid_accessor.setValueOnly(p, value)
                    else:
                        pass
        
        print(i, "/", len(edges))
    return tem_distance_grid, bounding_box_grid

distance_grid, bounding_box_grid = precompute_distances(points, edges,edge_radius, edge_bounding_boxes, vdb_grid)

50175 / 50176


In [187]:
print(distance_grid.evalActiveVoxelBoundingBox())
print(bounding_box_grid.evalActiveVoxelBoundingBox())
print(distance_grid.evalMinMax())
print(bounding_box_grid.evalMinMax())

((8, 10, 3), (637, 509, 966))
((8, 10, 3), (637, 509, 966))
(0.0, 3.5687408447265625)
(0.0, 0.0)


In [188]:
points, triangles, quads = distance_grid.convertToPolygons(isovalue=0.03125, adaptivity=1)

In [189]:
print(points.shape)
print(quads.shape)
print(triangles.shape)
print(quads[1])
# using the points and quads generate a .obj file can be shown in blender
def generate_mesh_obj(points, triangles, quads):
    with open('../data/convolution_surface_mesh.obj', 'w') as f:
        for p in points:
            f.write('v %f %f %f\n' % (p[0], p[1], p[2]))
        for face in quads:
            f.write('f %d %d %d %d\n' % (face[0]+1, face[1]+1, face[2]+1, face[3]+1))
        for triangle in triangles:
            f.write('f %d %d %d\n' % (triangle[0]+1, triangle[1]+1, triangle[2]+1))
        f.close()

generate_mesh_obj(points, triangles, quads)

(1801787, 3)
(1439059, 4)
(726940, 3)
[20 18 17 19]


In [190]:
# np.save('gaussian_integral_table.txt', table)
# print(gauss(1))
print(np.linalg.norm(edges[3][0] - edges[3][1]))

6.0
